In [14]:
import pandas as pd
import numpy as np
caminho = "C:\\Users\\natal\\OneDrive\\Área de Trabalho\\Estudo Python\\SistemaRecomendacao\\ml-latest-small"
caminho_arquivo = caminho + "\\movies.csv"
filmes = pd.read_csv(caminho_arquivo)
filmes.columns = ["filmeId", "titulo", "genero"]
filmes = filmes.set_index("filmeId")

caminho_arquivo = caminho + "\\ratings.csv"
notas = pd.read_csv(caminho_arquivo)
notas.columns = ["usuarioId", "filmeId", "nota", "momento"]

In [ ]:
total_de_votos = notas["filmeId"].value_counts()
filmes['total_de_votos'] = total_de_votos

nota_medias = notas.groupby("filmeId").mean()["nota"]
filmes["nota_media"] = nota_medias

In [19]:
filmes_com_mais_de_cinquenta_votos = filmes.query("total_de_votos > 50")
filmes_com_mais_de_cinquenta_votos.sort_values("nota_media", ascending = False).head(10)

,titulo,genero,total_de_votos,nota_media
filmeId,,,,
318,"Shawshank Redemption, The (1994)",Crime|Drama,317.0,4.429022
858,"Godfather, The (1972)",Crime|Drama,192.0,4.289062
2959,Fight Club (1999),Action|Crime|Drama|Thriller,218.0,4.272936
1276,Cool Hand Luke (1967),Drama,57.0,4.271930
750,Dr. Strangelove or: How I Learned to Stop Worr...,Comedy|War,97.0,4.268041
904,Rear Window (1954),Mystery|Thriller,84.0,4.261905
1221,"Godfather: Part II, The (1974)",Crime|Drama,129.0,4.259690
48516,"Departed, The (2006)",Crime|Drama|Thriller,107.0,4.252336
1213,Goodfellas (1990),Crime|Drama,126.0,4.250000


In [15]:
def distancia_de_vetores(a,b):
    return np.linalg.norm(a - b)

In [20]:
def notas_do_usuario(usuario):
    notas_do_usuario = notas.query("usuarioId==%d" % usuario)
    notas_do_usuario = notas_do_usuario[["filmeId", "nota"]].set_index(["filmeId"])
    return notas_do_usuario

In [21]:
def distancia_de_usuarios(usuarioId1, usuarioId2):
    notas1 = notas_do_usuario(usuarioId1)
    notas2 = notas_do_usuario(usuarioId2)
    diferencas = notas1.join(notas2, lsuffix = "_esquerda", rsuffix = "_direita").dropna()
    distancia = distancia_de_vetores(diferencas['nota_esquerda'], diferencas['nota_direita'])
    return [usuarioId1, usuarioId2, distancia]

In [22]:
def distancia_de_todos(voce_id, numero_de_usuarios_a_analisar  = None):
    todos_os_usuarios = notas['usuarioId'].unique()
    if numero_de_usuarios_a_analisar:
        todos_os_usuarios = todos_os_usuarios[:numero_de_usuarios_a_analisar ]
    distancias = [distancia_de_usuarios(voce_id, usuario_Id) for usuario_Id in todos_os_usuarios]
    distancias = list(filter(None, distancias))
    distancias = pd.DataFrame(distancias, columns = ["você", "outra_pessoa", "distancia"])
    return distancias

In [23]:
notas = notas.set_index("filmeId").loc[filmes_com_mais_de_cinquenta_votos.index]

In [24]:
notas = notas.reset_index()
notas.head()

,filmeId,usuarioId,nota,momento
0,1,1,4.0,964982703
1,1,5,4.0,847434962
2,1,7,4.5,1106635946
3,1,15,2.5,1510577970
4,1,17,4.5,1305696483


In [25]:
def knn(voce_id, k_mais_proximos=10, numero_de_usuarios_a_analisar = None):
  distancias = distancia_de_todos(voce_id, numero_de_usuarios_a_analisar = numero_de_usuarios_a_analisar)
  distancias = distancias.sort_values("distancia")
  distancias = distancias.set_index("outra_pessoa").drop(voce_id, errors='ignore')
  return distancias.head(k_mais_proximos)
     

In [27]:
def sugere_para(voce, k_mais_proximos = 10, numero_de_usuarios_a_analisar = None):
  notas_de_voce = notas_do_usuario(voce)
  filmes_que_voce_ja_viu = notas_de_voce.index

  similares = knn(voce, k_mais_proximos = k_mais_proximos, numero_de_usuarios_a_analisar = numero_de_usuarios_a_analisar)
  usuarios_similares = similares.index
  notas_dos_similares = notas.set_index("usuarioId").loc[usuarios_similares]
  recomendacoes = notas_dos_similares.groupby("filmeId").mean()[["nota"]]
  aparicoes = notas_dos_similares.groupby("filmeId").count()[['nota']]
  
  filtro_minimo = k_mais_proximos / 2
  recomendacoes = recomendacoes.join(aparicoes, lsuffix="_media_dos_usuarios", rsuffix="_aparicoes_nos_usuarios")
  recomendacoes = recomendacoes.query("nota_aparicoes_nos_usuarios >= %.2f" % filtro_minimo)  
  recomendacoes = recomendacoes.sort_values("nota_media_dos_usuarios", ascending=False)
  recomendacoes = recomendacoes.drop(filmes_que_voce_ja_viu,errors='ignore')
  return recomendacoes.join(filmes)

In [28]:
sugere_para(1, numero_de_usuarios_a_analisar=500)

,nota_media_dos_usuarios,nota_aparicoes_nos_usuarios,titulo,genero,total_de_votos,nota_media
filmeId,,,,,,
